In [1]:
import cv2
import torch
from PIL import Image
import depth_pro
import numpy as np
import matplotlib.pyplot as plt

c:\Users\one1c\.conda\envs\depth-pro\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check for GPU

torch.backends.cudnn.benchmark = True
torch.cuda.synchronize()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model and preprocessing transform
model, transform = depth_pro.create_model_and_transforms()
model.to(device)
model.half()
model.eval()

Using device: cuda


C:\Users\one1c\ml-depth-pro_video\src\depth_pro\depth_pro.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(config.checkpoint_uri, map_location="

DepthPro(
  (encoder): DepthProEncoder(
    (patch_encoder): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linea

In [3]:
def process_frame(frame):
    """Process a single video frame to predict depth."""
    # Convert frame to PIL Image
    frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Apply the transform directly
    image_tensor = transform(frame_image).to(device).half()  # Convert input to FP16
    
    # Infer depth
    with torch.no_grad():
        prediction = model.infer(image_tensor)
    
    # Get depth map
    depth = prediction["depth"].to(device)
    
    # Normalize depth for visualization
    depth_normalized = (depth - depth.min()) / (depth.max() - depth.min())
    # print(depth_normalized)
    depth_numpy = depth_normalized.cpu().numpy()
    
    return depth_numpy

In [8]:
def live_depth_feed():
    """Capture and process live video feed from webcam."""
    cap = cv2.VideoCapture(0)  # Open webcam (index 0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break
        
        # Process the frame to get depth
        depth_map = process_frame(frame)
        
        # Normalize and convert depth map to 8-bit for visualization
        depth_visual = (depth_map * 255).astype(np.uint8)
        depth_colormap = cv2.applyColorMap(depth_visual, cv2.COLORMAP_JET)
        
        # Display the depth map
        cv2.imshow("Depth Map", depth_colormap)
        
        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


In [9]:
live_depth_feed()

In [1]:
ffmpeg -i video_test/video_test_depth.mp4 -vf "fps=10,scale=600:-1" video_test/video_test_depth.gif


SyntaxError: invalid syntax (1689307125.py, line 1)